In [1]:
# To push updates:
#  git add *.ipynb *.py
#  git commit -m "Update notebook"
#  git push


In [2]:
# %% 0. SET-UP  ──────────────────────────────────────────────────────────────
# Widget backend (allows live zoom & re-draw);  use  %matplotlib inline
# if ipympl is not available.
%matplotlib widget          

import matplotlib, matplotlib.pyplot as plt
matplotlib.rcParams['animation.html']      = 'jshtml'
matplotlib.rcParams['animation.embed_limit'] = 100  # MB

import networkx as nx, itertools, time, threading
import ipywidgets as w
from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter
from IPython.display import HTML, display, clear_output


In [3]:
# %% 1. GRAPH & COLOURING HELPERS ───────────────────────────────────────────
def build_n_pan(n: int) -> nx.Graph:
    """Return the n-pan graph (cycle C_n + one pendant)."""
    G = nx.cycle_graph(range(1, n + 1))
    G.add_node(0); G.add_edge(0, 1)
    return G

def base_colouring(n: int) -> dict[int, int]:
    """Equitable χ₌-colouring of C⁺¹_n (2 colours if n even, 3 if n odd)."""
    col = {}
    if n % 2 == 0:                   # even → 2-colour
        for i in range(1, n + 1): col[i] = 1 if i % 2 else 2
        col[0] = 2
    else:                            # odd → 3-colour with mod-3 pattern
        r = n % 3
        if r == 0:
            for i in range(1, n + 1): col[i] = (i-1) % 3 + 1
            col[0] = 3
        elif r == 1:
            for i in range(1, n):     col[i] = (i-1) % 3 + 1
            col[n], col[0] = 2, 3
        else:  # r == 2
            for i in range(1, n-1):   col[i] = (i-1) % 3 + 1
            col[n-1], col[n], col[0] = 1, 2, 3
    return col                        # values are 1‒k

def coloured_floor(n: int, m: int):
    """Return (G×P_m, colour_map dict, k) with layer-shift colouring."""
    base   = base_colouring(n)
    k      = max(base.values())
    GH     = nx.cartesian_product(build_n_pan(n), nx.path_graph(range(1, m + 1)))
    colour = {(u, v): ((base[u] + v-1) % k or k) for u, v in GH.nodes()}
    return GH, colour, k


In [4]:
# %% 2. STATIC PREVIEW (sliders + Run button) ───────────────────────────────
n_slider = w.IntSlider(value=5, min=3,  max=15, description='n')
m_slider = w.IntSlider(value=4, min=2,  max=12, description='m')
run_btn  = w.Button(description='Show Floor Graph')
preview  = w.Output()

def _preview(_):
    with preview:
        clear_output(wait=True)
        n, m               = int(n_slider.value), int(m_slider.value)
        G, cmap, k         = coloured_floor(n, m)
        colours            = ['grey', 'red', 'green', 'blue'][:k+1]
        fig, ax            = plt.subplots(figsize=(6, 5))
        nx.draw(G, nx.spring_layout(G, seed=42),
                node_color=[colours[c] for c in cmap.values()],
                with_labels=False, ax=ax)
        ax.set_title(f"Full colouring, n={n}, m={m}, k={k}")
        plt.show()

run_btn.on_click(_preview)
display(w.VBox([n_slider, m_slider, run_btn, preview]))


In [11]:
# %% 4. CLASSROOM ANIMATION (one colour per frame) ─────────────────────────
def classroom_anim(n: int, m: int) -> FuncAnimation:
    G, cmap, k   = coloured_floor(n, m)
    pos          = nx.spring_layout(G, seed=42)
    colours      = ['lightgrey', 'red', 'green', 'blue']          # up to k=3
    frames       = list(range(1, k + 1)) + [k + 1]               # last = all
    fig, ax      = plt.subplots(figsize=(6, 5))

    def draw(frame):
        ax.clear()
        if frame <= k:
            visible = {node for node, c in cmap.items() if c == frame}
            node_c  = [colours[c] if node in visible else colours[0] for node, c in cmap.items()]
            title   = f"Step {frame}: show colour {frame}"
        else:                                                   # final frame
            node_c  = [colours[c] for c in cmap.values()]
            title   = "Step final: all colours"
        nx.draw(G, pos, node_color=node_c, with_labels=False, ax=ax)
        ax.set_title(title)

    return FuncAnimation(fig, draw, frames=frames, interval=800, repeat=False)

# --- small helper widget ---------------------------------------------------
save_btn  = w.Button(description='Save MP4', button_style='info')
anim_out  = w.Output()

def _run_anim(_):
    with anim_out:
        clear_output()
        ani = classroom_anim(int(n_slider.value), int(m_slider.value))
        display(HTML(ani.to_jshtml()))
        save_btn.ani = ani              # keep a reference for export

def _save_mp4(_):
    with anim_out:
        if not hasattr(save_btn, 'ani'): print("Run the animation first."); return
        n, m   = int(n_slider.value), int(m_slider.value)
        fname  = f"pan_path_n{n}_m{m}.mp4"
        try:
            save_btn.ani.save(fname, writer=FFMpegWriter(fps=1, bitrate=1800))
            print("✔ Saved MP4 →", fname)
        except Exception as e:
            print("⚠ MP4 failed, saving GIF instead:", e)
            gif = fname.replace(".mp4", ".gif")
            save_btn.ani.save(gif, writer=PillowWriter(fps=1))
            print("✓ Saved GIF →", gif)

run_anim_btn = w.Button(description='Run Animation')
run_anim_btn.on_click(_run_anim)
save_btn.on_click(_save_mp4)

display(w.VBox([run_anim_btn, save_btn, anim_out]))


In [6]:
# A.  silent set-up: create sliders + console + log(); show nothing
import ipywidgets as w
from IPython.display import display

n_val = int(n_slider.value)
m_val = int(m_slider.value)

disp_out = w.Output()                
def log(msg:str):
    """Thread-safe print into disp_out (no buffering issues)."""
    disp_out.append_stdout(str(msg) + "\n")



In [7]:
import threading, itertools, time, traceback

class Dispatcher(threading.Thread):
    def __init__(self, groups:dict[int,list], slot:int=10):
        super().__init__(daemon=True)
        self.groups, self.slot = groups, slot
        self._stop_ev = threading.Event()

    def stop(self):      self._stop_ev.set()
    def is_alive(self):  return super().is_alive() and not self._stop_ev.is_set()

    def run(self):
        try:
            k        = len(self.groups)
            cycle    = itertools.cycle(range(1, k+1))
            prev_gen = None

            while not self._stop_ev.is_set():
                cur_gen = next(cycle)
                ts      = time.strftime("[%H:%M:%S]")

                if prev_gen is None:
                    log(f"{ts}  Generator {cur_gen} ON   ({len(self.groups[cur_gen])} lamps)")
                else:
                    log(f"{ts}  Generator {prev_gen} OFF | "
                        f"Generator {cur_gen} ON   ({len(self.groups[cur_gen])} lamps)")

                prev_gen = cur_gen
                time.sleep(self.slot)

        except Exception:
            log("\nDispatcher crashed:")
            log(traceback.format_exc())


In [8]:
import networkx as nx

def build_n_pan(n:int):
    G = nx.cycle_graph(range(1, n+1))
    G.add_node(0); G.add_edge(0, 1)
    return G

def compute_base_color(n:int):
    base={}
    if n%2==0:
        for i in range(1,n+1): base[i]=1 if i%2 else 2
        base[0]=2
    else:
        r=n%3
        if r==0:  base[0]=3;   base.update({i:((i-1)%3)+1 for i in range(1,n+1)})
        elif r==1:
            base[0]=3
            base.update({i:((i-1)%3)+1 for i in range(1,n)})
            base[n]=2
        else:
            base[0]=3
            base.update({i:((i-1)%3)+1 for i in range(1,n-1)})
            base[n-1]=1; base[n]=2
    return base

def make_dispatcher_groups(n:int,m:int):
    G = nx.cartesian_product(build_n_pan(n), nx.path_graph(range(1,m+1)))
    base = compute_base_color(n)
    k    = max(base.values())
    colour = {(u,v):((base[u]+v-1)%k or k) for u,v in G.nodes()}
    groups={g:[] for g in range(1,k+1)}
    for node,c in colour.items(): groups[c].append(node)
    return groups,k


In [9]:
import ipywidgets as w
from IPython.display import display

start_btn = w.Button(description='Start Dispatcher', button_style='success')
stop_btn  = w.Button(description='Stop  Dispatcher', button_style='danger')
dispatcher = None           # global reference

def _start(_):
    global dispatcher
    if dispatcher and dispatcher.is_alive():
        dispatcher.stop()

    disp_out.clear_output()

    # read current slider values *without* showing them up top
    n_val, m_val = int(n_slider.value), int(m_slider.value)
    groups, _ = make_dispatcher_groups(n_val, m_val)

    log("Lamp count per generator")
    for g,L in groups.items():
        log(f"  Generator {g}: {len(L)} lamps")

    dispatcher = Dispatcher(groups, slot=10)   # 10-second slot for demo
    dispatcher.start()
    log("\nDispatcher started …")

def _stop(_):
    if dispatcher and dispatcher.is_alive():
        dispatcher.stop()
        log("\nDispatcher stopped.")

start_btn.on_click(_start)
stop_btn .on_click(_stop)

# layout = buttons first, then console; sliders line optional
ui = w.VBox([
        w.HBox([start_btn, stop_btn]),
        disp_out,
        w.HBox([n_slider, m_slider])   # ← delete this line if you never want to see sliders
     ])

display(ui)
